# Get Data

In [1]:
import folium
import pandas as pd, numpy as np
import re, decimal, boto3, os

#### Get the data from Wikipedia

In [2]:
rawdata = pd.read_html("https://en.wikipedia.org/wiki/List_of_the_highest_major_summits_of_Colorado"
                       , match='highest summits', header=0)[0]
df = rawdata.drop(['Rank'], axis=1)
df = df.astype(str)
df.head()

,Mountain Peak,Mountain Range,Elevation,Prominence,Isolation,Location
0,Mount Elbert[1][2][3][4][5],Sawatch Range,4401.2 m,2772 m,nan,39°07′04″N 106°26′43″W﻿ / ﻿39.1178°N 106.4454°W
1,Mount Massive[6][7][8][9][10][11],Sawatch Range,4398 m,598 m,8.14 km,39°11′15″N 106°28′33″W﻿ / ﻿39.1875°N 106.4757°W
2,Mount Harvard[12][13][14][15][16][17],Sawatch Range,4395.6 m,719 m,24 km,38°55′28″N 106°19′15″W﻿ / ﻿38.9244°N 106.3207°W
3,Blanca Peak[18][19][20][21][22][23],Sangre de Cristo Mountains,4374 m,1623 m,166.4 km,37°34′39″N 105°29′08″W﻿ / ﻿37.5775°N 105.4856°W
4,La Plata Peak[24][25][26][27],Sawatch Range,4372 m,560 m,10.11 km,39°01′46″N 106°28′22″W﻿ / ﻿39.0294°N 106.4729°W


### Format the data

#### reformat column names

In [3]:
df.columns = [re.sub('\ ', '_', col) for col in df.columns]

#### remove citations

In [4]:
rexp = re.compile('\[\d+\]')
df['Mountain_Peak'] = df['Mountain_Peak'].replace(rexp, '')

#### only want float format

In [5]:
rexp = re.compile('([\d]+\.[\d]+)')
df['Latitude'] = df['Location'].apply(lambda x: re.findall(rexp, x)[0])
df['Longitude'] = df['Location'].apply(lambda x: re.findall(rexp, x)[1])
df = df.drop(['Location'], axis=1)

#### represent units in column name

In [6]:
#record before stripping
unitmapper = {'Elevation': 'm', 'Prominence': 'm', 'Isolation': 'km'}
#strip out from values
for col in unitmapper.keys():
    df[col] = df[col].apply(lambda s: re.sub('\sk?m', '', s))
df.rename({key: key+'_'+val for (key, val) in unitmapper.items()}, axis=1, inplace=True)
unit_cols = [key+'_'+val for (key, val) in unitmapper.items()]

#### index

In [7]:
df = df.set_index(['Mountain_Peak'])

#### fill missing values

In [8]:
#source: https://www.peakbagger.com/Isolate.aspx?pid=5736
##seems way high but who am i to say
df.loc['Mount Elbert']['Isolation_km'] = '1079.15'

#### reformat column datatypes

In [9]:
#units
decimal.getcontext().prec = 2
for col in unit_cols:
    df[col] = df[col].astype('float').apply(lambda x: round(x, 2))
#coordinates
df['Latitude'] = df['Latitude'].astype('float')
df['Longitude'] = df['Longitude'].astype('float')
##these are backwards
df['Longitude'] = df['Longitude']*-1

# Map Data

In [10]:
def create_map():
    #geographical center of state
    latitude, longitude = 39.000000, -105.547222
    mapobj = folium.Map(location=[latitude, longitude], zoom_start=7)
    return mapobj
map_co = create_map()

In [11]:
df = pd.read_csv("https://davidschneiderprojects.s3.amazonaws.com/MountainMapping/data.csv")
df = df.set_index('Mountain_Peak')

#### Add markers to map

In [12]:
for mountain, data in df.iterrows():
    label = folium.Popup(mountain, parse_html=True)
    marker = folium.CircleMarker(
        [data['Latitude'], data['Longitude']],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False)
    marker.add_to(map_co)

#### Render the map

In [13]:
map_co